In [1]:
# import statements
import numpy as np
import copy
from ipynb.fs.full.data_explore import DataExplorer
from collections import Counter
import pprint

In [2]:
class FunctionPrediction():
    def __init__(self, organism_name):
        # get all the annotation, adjacency list data from the explorer class
        self.data_explorer = DataExplorer(organism_name)
        
    def get_nborhood(self, protein, r, df="set"):
        # to get r-neighborhood of protein
        if r == 1:
            fnl_nborhood = [
                nbor for (nbor, score) in self.data_explorer.adj_list[protein]
            ]

            if df == "set":
                return set(fnl_nborhood), set(fnl_nborhood)
            else:
                return fnl_nborhood, fnl_nborhood

        accum, last_nborhood = self.get_nborhood(protein, r - 1, df)

        fnl_nborhood = []

        for protein in last_nborhood:
            for (nbor, score) in self.data_explorer.adj_list[protein]:
                fnl_nborhood.append(nbor)

        if df == "set":
            return set.union(set(accum), set(fnl_nborhood)), set(fnl_nborhood)
        else:
            return accum + fnl_nborhood, fnl_nborhood
        
    def majority_rule(self, protein_id_list, r=1):
        """
        protein_id_list: list of proteins with unknown function you want to predict
        r: parameter for radius of neighborhood to consider
        """
        # input: protein list, adjacency list graph representation
        # output: cluster assignment for each protein in list based on the majority rule of protein's neighbors
        
        
        # this was me trying to come up with a way to get specific labels for clusters but then realizing
        # a paper from 2001 did something that i liked better so i used that below, kept it anyway tho
        def tsmallest_cluster_label(protein_id):
            print("{} has {} neighbors".format(protein_id, len(self.data_explorer.adj_list[protein_id])))
        
            tmp = [
                self.data_explorer.annotation_list[nbor[0]]
                for nbor in self.data_explorer.adj_list[protein_id]
            ]

            nbor_clusters = []

            # need to identify this neighbor's cluster by membership to t^th-smallest cluster, parameterized
            for cluster_list in tmp:
                # (cluster, size) tuples, sort by size
                zipped_list = [
                    (cluster_id, self.data_explorer.cluster_sizes[cluster_id])
                    for cluster_id in cluster_list
                ]

                sorted_list = sorted(zipped_list, key=lambda tup: tup[1])

                # for each neighbor, we take the t^th smallest cluster that it's a part of, assign that to be
                # its identity, then add this cluster to the neighbor cluster list from which we pick the majority
                nbor_clusters.append(
                    sorted_list[min(t - 1, len(sorted_list) - 1)][0]
                )

            most_common_nbor = Counter(nbor_clusters).most_common(1)[0][0]
            
            return most_common_nbor
        
        def expected_label_weight(protein_id, cluster_id):
            # e(v, d, a) = expected number of nodes in a neighborhood of size n(v, d) with function a
            # calculated as (# neighbors) * (# nodes labeled cluster_id / # nodes in network)
            return len(self.data_explorer.adj_list[protein_id]) * (
                self.data_explorer.cluster_sizes[cluster_id] / len(self.data_explorer.adj_list)
            )
        
        def hishigaki_label(protein_id, n=15):
            # want to maximize the function:
            # |# labeled cluster_id - # expected labeled cluster_id| / # expected labeled cluster_id
            # so we are taking the argmax of # labeled cluster_id over all function labels
            label_score_dict = {}
            
            total_nbh, recent_nbh = self.get_nborhood(protein_id, r, df="list")
        
            clusters = []
            for nbor in total_nbh:
                clusters += self.data_explorer.annotation_list[nbor]
                
            cluster_counts = dict(Counter(clusters))
            
            for cluster in self.data_explorer.cluster_sizes:
                e = expected_label_weight(protein_id, cluster)

                try:
                    f = cluster_counts[cluster]
                except KeyError:
                    f = 0
                
                try:
                    label_score_dict[cluster] = ((f - e ) ** 2) / e
                except:
                    label_score_dict[cluster] = 0
                        
            for cluster in label_score_dict:
                if cluster not in cluster_counts:
                    label_score_dict[cluster] = 0
                else:
                    label_score_dict[cluster] = label_score_dict[cluster] * \
                                                1 #(cluster_scores[cluster] / cluster_counts[cluster])
            
            zipped_dict = [(cluster, label_score_dict[cluster]) for cluster in label_score_dict]
            zipped_dict_sorted = sorted(zipped_dict, key = lambda tup: tup[1])

            
            winners = ([cluster for (cluster, score) in zipped_dict_sorted][::-1])
            winners_scores = [label_score_dict[i] for i in winners]
            
            # winners are returned in reverse sorted order in terms of score, so highest scoring function is at
            # index 0 and descends from there
            return winners
        
        cluster_assignments = {}
        
        for protein in protein_id_list:
            cluster_assignments[protein] = hishigaki_label(protein)
        
        return cluster_assignments
            
    def functional_flow(self, protein_id_list, t=2):
        """
        protein_id_list: list of proteins with unknown function you want to predict
        t: parameter for number of iterations of functional flow; also determines radius of flow from each
        unknown protein
        """
        def expected_label_weight(protein_id, cluster_id):
            # e(v, d, a) = expected number of nodes in a neighborhood of size n(v, d) with function a
            # calculated as (# neighbors) * (# nodes labeled cluster_id / # nodes in network)
            return len(self.data_explorer.adj_list[protein_id]) * (
                self.data_explorer.cluster_sizes[cluster_id] / len(self.data_explorer.adj_list)
            )
    
        working_nborhood = set([])
        for protein_id in protein_id_list:
            total_nbh, recent_nbh = self.get_nborhood(protein_id, t, df="set")
            working_nborhood.update(total_nbh)
    
        
        print("WORKING NBORHOOD SIZE:", len(working_nborhood))
        
        # input: protein_id, adjacency list graph representation
        # output: cluster_id of protein_id based on the majority rule of protein_id's neighbors
        
        # idea from Nabieva et al paper:
            # Whole-proteome prediction of protein function via graph-theoretic analysis of interaction maps
        
        # we start by picking some list of proteins that we want to know the function of
        # we also start with some known protein functions of nodes in the network
        # we fill the known nodes with infinite water at t=0
        # for t timesteps, we simulate the flow of water out of the known nodes, along edges, to proteins of
        # unknown function, and at t=t, we look at how much water has reached unknown nodes
        # (the flow volume between nodes is determined by edge weight)
        
        resoviors = {0: {}} # fill the nodes at t=0 according to known function or not
        # resoviors keep track of resoviors at each timestep
        # at a given timestep i, for each node u, we track the amount in the resovior for function a that u has
        
        print("INIT RESOVIORS")
        for protein in self.data_explorer.adj_list:
            if protein not in protein_id_list:
                function_dict = {}
                for cluster in self.data_explorer.cluster_sizes:
                    if cluster in self.data_explorer.annotation_list[protein]:
                        function_dict[cluster] = 2 ** 100
                    else:
                        function_dict[cluster] = 0.0
                
                resoviors[0][protein] = function_dict
            else:
                function_dict = {}
                for cluster in self.data_explorer.cluster_sizes:
                    function_dict[cluster] = 0.0
                
                resoviors[0][protein] = function_dict
                        
        def edge_capacity(u, v, a, t):
            cluster_size = self.data_explorer.cluster_sizes[a]
            
            if t == 0: return 0
            if resoviors[t - 1][u][a] <= resoviors[t - 1][v][a]:
                return 0
            else:
                uv_weight = 0
                for (protein, score) in self.data_explorer.adj_list[u]:
                    if protein == v:
                        uv_weight = score
                
                u_weights = sum([score for (_, score) in self.data_explorer.adj_list[u]])
                
                return min(
                    uv_weight,
                    (uv_weight / u_weights)
                ) / cluster_size
       
        # initialize the label scoring dictionary
        label_score_dict = {}
        for protein in protein_id_list:
            label_score_dict[protein] = {}
            for cluster in self.data_explorer.cluster_sizes:
                label_score_dict[protein][cluster] = 0.0
        
        # build the resoviors
        print("----- FF SIMULATION -----")
        
        for i in range(1, t + 1):
            print("ITER: {} / {}".format(i, t))
            resovior_i = copy.deepcopy(resoviors[i - 1])
            for protein in working_nborhood:
                for cluster in self.data_explorer.cluster_sizes:
                    test_val = sum(
                        [ edge_capacity(nbor, protein, cluster, i) - edge_capacity(protein, nbor, cluster, i)
                          for (nbor, score) in self.data_explorer.adj_list[protein] ]
                    )
                    resovior_i[protein][cluster] = resoviors[i - 1][protein][cluster] + test_val
                    
            resoviors[i] = resovior_i
            
            for protein in protein_id_list:
                for cluster in self.data_explorer.cluster_sizes:
                    for (nbor, score) in self.data_explorer.adj_list[protein]:
                        label_score_dict[protein][cluster] += edge_capacity(nbor, protein, cluster, i)
        
        # determine winning clusters for unknown proteins
        cluster_assignments_dict = {}
        
        for protein in protein_id_list:
            cluster_assignments_dict[protein] = []
            for cluster in label_score_dict[protein]:
                if label_score_dict[protein][cluster] > 0: # add all non-zero flow scores
                    cluster_assignments_dict[protein].append((cluster, label_score_dict[protein][cluster]))
        
        cluster_assignments = {}
        
        for protein in cluster_assignments_dict:
            score_list = cluster_assignments_dict[protein]
            sorted_score_list = [cluster for (cluster, score) in sorted(score_list, key = lambda tup: tup[1])]
            cluster_assignments[protein] = sorted_score_list[::-1]
        
        # winners are returned in reverse sorted order in terms of score, so highest scoring function is at
        # index 0 and descends from there
        return cluster_assignments

### Usage of ``FunctionPrediction`` class:
To create a new instance of the class, create a new variable with the organism name (same as organism name used to instantiate the DataExplorer class; will be the name of the folder which contains the PPI info):
```
function_predictor = FunctionPrediction("organism_name")
```

To perform the majority approach with a radius `r` and list of proteins to predict `p`, run the function:
```
majority_approach_results = function_predictor.majority_rule(p, r=r)
```

To perform functiona flow for `t` iterations and list of proteins to predict `p`, run the function:
```
functional_flow_results = function_predictor.functional_flow(p, t=t)
```

In both methods, the function returns a dictionary of the form:
```
{protein_id: [list of functions (by name) scored and sorted in descending order]}
```